# RAG app Evaluation
In this use case it is shown how to extract information from a PDF file through LLM queries with RAG (Retrieval Augmented Generation) technology. For this use case is necessary the use of OpenAI embeddings and model API calls. To show the final result, the model is embedded on a Gradio UI. In this version it would be used Llama Index as vector databse.

In this notebook, it will be conducted an evaluation on the v4 version of the RAG system. This evaluation will consist on evaluating the following concepts:

* Answer relevance
* Context Relevance
* Groundedness

For this evaluation, a set of evaluation questions has been selected in order to make the system align with RAG system purposes. Along eith these questions, several possible reasons that cover different topics a  should cover to improve answers and value in them. 

## Model Evalutation using Llama Index

In [1]:
# Env keys library
from dotenv import load_dotenv

# Standard RAG libraries
from llama_index import SimpleDirectoryReader
from llama_index import Document
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.embeddings import OpenAIEmbedding

# Advanced RAG libraries
from llama_index import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index import load_index_from_storage

# Common libraries
import pandas as pd
import os

# Set env keys and global variables
load_dotenv("apis.env")

# Select pdf with text
pdf = "./attention-is-all-you-need.pdf"

# Configure LLM hyperparameters
temperature = 0

### Evaluation set loading & Tru Object Instance

In [2]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        # print(item)
        eval_questions.append(item)

# Instanciate Tru object
from trulens_eval import Tru
tru = Tru()

# Initiate Tru app
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


### 1 - Standard RAG

In [3]:
def evaluation_responses(pdf, temperature):
    documents = SimpleDirectoryReader(
        input_files=[pdf]
    ).load_data()
    document = Document(text="\n\n".join([doc.text for doc in documents]))
    # embed_model = OpenAIEmbedding(model="text-embedding-ada-002", embed_batch_size=10)
    embed_model = "local:BAAI/bge-small-en-v1.5"
    llm = OpenAI(model="gpt-3.5-turbo", temperature=temperature)#streaming=True)
    
    service_context = ServiceContext.from_defaults(
        llm=llm, embed_model=embed_model
    )
    index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)
    query_engine = index.as_query_engine()
    # response = query_engine.query(msg)
    return query_engine

query_engine = evaluation_responses(pdf, temperature=temperature)#, max_tokens=max_tokens

#### 1.1 - RAG Triad Feedback

In [4]:
from trulens_eval import (
    Feedback,
    TruLlama,
    OpenAI
)
from trulens_eval.feedback import Groundedness
import numpy as np

openai = OpenAI()

qa_relevance = (
    Feedback(openai.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input_output()
)

qs_relevance = (
    Feedback(openai.relevance_with_cot_reasons, name = "Context Relevance")
    .on_input()
    .on(TruLlama.select_source_nodes().node.text)
    .aggregate(np.mean)
)

grounded = Groundedness(groundedness_provider=openai)

groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name="Groundedness")
        .on(TruLlama.select_source_nodes().node.text)
        .on_output()
        .aggregate(grounded.grounded_statements_aggregator)
)

feedbacks = [qa_relevance, qs_relevance, groundedness]

def get_prebuilt_trulens_recorder(query_engine, app_id):
    tru_recorder = TruLlama(
        query_engine,
        app_id=app_id,
        feedbacks=feedbacks
        )
    return tru_recorder

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [5]:
# Standard RAG Pipeline
tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Standard RAG")

In [6]:
# Evaluate standard RAG version with questions
emo = "\U00002705"
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)
        print(emo+" "+question)

✅ What is the main theme or topic of this document?
✅ Can you summarize the key points made in this document?
✅ What are the primary arguments or claims presented?
✅ Are there any notable facts or figures mentioned? If so, what are they?
✅ Can you provide an example or case study mentioned in the document?
✅ What conclusions are drawn in this document?
✅ Are there any contradictions or points of debate within the document?
✅ What background or contextual information is provided?
✅ What are the limitations or gaps identified in the document?
✅ What recommendations or next steps does the document propose?


In [7]:
# Extract results of evaluation
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [8]:
# Vizualize evaluation results on each concept
# tru.run_dashboard()

In [9]:
# tru.stop_dashboard()

## Advanced RAG

#### Setup functions

In [10]:
def build_sentence_window_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index",
):
    documents = SimpleDirectoryReader(
        input_files=[pdf]
    ).load_data()
    document = Document(text="\n\n".join([doc.text for doc in documents]))

    # Create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            [document], service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index


def get_sentence_window_query_engine(
    sentence_index, similarity_top_k=6, rerank_top_n=2
):
    # Define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

def run_evals(eval_questions, tru_recorder, query_engine):
    emo = "\U00002705"
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)
            print(emo+" "+question)

### 2 - Sentence Window s=1

In [11]:
from llama_index.llms import OpenAI

sentence_index_1 = build_sentence_window_index(
    pdf,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0),
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=1,
    save_dir="sentence_index_1",
)

sentence_window_engine_1 = get_sentence_window_query_engine(
    sentence_index_1
)

tru_recorder_1 = get_prebuilt_trulens_recorder(
    sentence_window_engine_1,
    app_id='Sentence Window s=1'
)

run_evals(eval_questions, tru_recorder_1, sentence_window_engine_1)

✅ What is the main theme or topic of this document?
✅ Can you summarize the key points made in this document?
✅ What are the primary arguments or claims presented?
✅ Are there any notable facts or figures mentioned? If so, what are they?
✅ Can you provide an example or case study mentioned in the document?
✅ What conclusions are drawn in this document?
✅ Are there any contradictions or points of debate within the document?
✅ What background or contextual information is provided?
✅ What are the limitations or gaps identified in the document?
✅ What recommendations or next steps does the document propose?


### 3 - Sentence Window s=2

In [12]:
# from llama_index.llms import OpenAI

sentence_index_2 = build_sentence_window_index(
    pdf,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0),
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=2,
    save_dir="sentence_index_2",
)

sentence_window_engine_2 = get_sentence_window_query_engine(
    sentence_index_2
)

tru_recorder_2 = get_prebuilt_trulens_recorder(
    sentence_window_engine_2,
    app_id='Sentence Window s=2'
)

run_evals(eval_questions, tru_recorder_2, sentence_window_engine_2)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1b557257050 is calling an instrumented method <function BaseQueryEngine.query at 0x000001B5371F1580>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1b54d543a50) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1b557257050 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x000001B5390C5620>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1b54d543a50) using this function.
A new object of type <class 'llama_index.indices.vector_store.retrievers.retriever.VectorIndexRetriever'> at 0x1b557256f10 is calling an instrumented method <function BaseRetriever.retrieve at 0x000001B5371F04A0>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on ot

✅ What is the main theme or topic of this document?


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1b557257050 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x000001B5390C5620>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1b54d543a50) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b557256f50 is calling an instrumented method <function CompactAndRefine.get_response at 0x000001B53725C5E0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x1b54d543a90) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b557256f50 is calling an instrumented method <function Refine.get_response at 0x000001B537B57E20>. The path of this call may be incorrect.
Guessing path of new object is ap

✅ Can you summarize the key points made in this document?


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1b557257050 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x000001B5390C5620>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1b54d543a50) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b557256f50 is calling an instrumented method <function CompactAndRefine.get_response at 0x000001B53725C5E0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x1b54d543a90) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b557256f50 is calling an instrumented method <function Refine.get_response at 0x000001B537B57E20>. The path of this call may be incorrect.
Guessing path of new object is ap

...

✅ What background or contextual information is provided?


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1b557257050 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x000001B5390C5620>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1b54d543a50) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b557256f50 is calling an instrumented method <function CompactAndRefine.get_response at 0x000001B53725C5E0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x1b54d543a90) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b557256f50 is calling an instrumented method <function Refine.get_response at 0x000001B537B57E20>. The path of this call may be incorrect.
Guessing path of new object is ap

✅ What are the limitations or gaps identified in the document?


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1b557257050 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x000001B5390C5620>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1b54d543a50) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b557256f50 is calling an instrumented method <function CompactAndRefine.get_response at 0x000001B53725C5E0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x1b54d543a90) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b557256f50 is calling an instrumented method <function Refine.get_response at 0x000001B537B57E20>. The path of this call may be incorrect.
Guessing path of new object is ap

✅ What recommendations or next steps does the document propose?


### 4 - Sentence Window s=3

In [13]:
# from llama_index.llms import OpenAI

sentence_index_3 = build_sentence_window_index(
    pdf,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0),
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index_3",
)

sentence_window_engine_3 = get_sentence_window_query_engine(
    sentence_index_3
)

tru_recorder_3 = get_prebuilt_trulens_recorder(
    sentence_window_engine_3,
    app_id='Sentence Window s=3'
)

run_evals(eval_questions, tru_recorder_3, sentence_window_engine_3)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1b54d536ad0 is calling an instrumented method <function BaseQueryEngine.query at 0x000001B5371F1580>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1b54d543a50) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1b54d536ad0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x000001B5390C5620>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1b54d543a50) using this function.
A new object of type <class 'llama_index.indices.vector_store.retrievers.retriever.VectorIndexRetriever'> at 0x1b54d535390 is calling an instrumented method <function BaseRetriever.retrieve at 0x000001B5371F04A0>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on ot

✅ What is the main theme or topic of this document?


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1b54d536ad0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x000001B5390C5620>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1b54d543a50) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b54d537710 is calling an instrumented method <function CompactAndRefine.get_response at 0x000001B53725C5E0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x1b54d543a90) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b54d537710 is calling an instrumented method <function Refine.get_response at 0x000001B537B57E20>. The path of this call may be incorrect.
Guessing path of new object is ap

✅ Can you summarize the key points made in this document?


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1b54d536ad0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x000001B5390C5620>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1b54d543a50) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b54d537710 is calling an instrumented method <function CompactAndRefine.get_response at 0x000001B53725C5E0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x1b54d543a90) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b54d537710 is calling an instrumented method <function Refine.get_response at 0x000001B537B57E20>. The path of this call may be incorrect.
Guessing path of new object is ap

...

✅ What background or contextual information is provided?


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1b54d536ad0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x000001B5390C5620>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1b54d543a50) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b54d537710 is calling an instrumented method <function CompactAndRefine.get_response at 0x000001B53725C5E0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x1b54d543a90) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b54d537710 is calling an instrumented method <function Refine.get_response at 0x000001B537B57E20>. The path of this call may be incorrect.
Guessing path of new object is ap

✅ What are the limitations or gaps identified in the document?


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1b54d536ad0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x000001B5390C5620>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1b54d543a50) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b54d537710 is calling an instrumented method <function CompactAndRefine.get_response at 0x000001B53725C5E0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x1b54d543a90) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b54d537710 is calling an instrumented method <function Refine.get_response at 0x000001B537B57E20>. The path of this call may be incorrect.
Guessing path of new object is ap

✅ What recommendations or next steps does the document propose?


### 4 - Sentence Window s=5

In [14]:
# from llama_index.llms import OpenAI

sentence_index_5 = build_sentence_window_index(
    pdf,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0),
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=5,
    save_dir="sentence_index_5",
)

sentence_window_engine_5 = get_sentence_window_query_engine(
    sentence_index_5
)

tru_recorder_5 = get_prebuilt_trulens_recorder(
    sentence_window_engine_5,
    app_id='Sentence Window s=5'
)

run_evals(eval_questions, tru_recorder_5, sentence_window_engine_5)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1b551265a90 is calling an instrumented method <function BaseQueryEngine.query at 0x000001B5371F1580>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1b557257050) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1b551265a90 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x000001B5390C5620>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1b557257050) using this function.
A new object of type <class 'llama_index.indices.vector_store.retrievers.retriever.VectorIndexRetriever'> at 0x1b551264bd0 is calling an instrumented method <function BaseRetriever.retrieve at 0x000001B5371F04A0>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on ot

✅ What is the main theme or topic of this document?


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1b551265a90 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x000001B5390C5620>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1b557257050) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b551265fd0 is calling an instrumented method <function CompactAndRefine.get_response at 0x000001B53725C5E0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x1b557256f50) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b551265fd0 is calling an instrumented method <function Refine.get_response at 0x000001B537B57E20>. The path of this call may be incorrect.
Guessing path of new object is ap

✅ Can you summarize the key points made in this document?


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1b551265a90 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x000001B5390C5620>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1b557257050) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b551265fd0 is calling an instrumented method <function CompactAndRefine.get_response at 0x000001B53725C5E0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x1b557256f50) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b551265fd0 is calling an instrumented method <function Refine.get_response at 0x000001B537B57E20>. The path of this call may be incorrect.
Guessing path of new object is ap

...

✅ What background or contextual information is provided?


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1b551265a90 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x000001B5390C5620>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1b557257050) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b551265fd0 is calling an instrumented method <function CompactAndRefine.get_response at 0x000001B53725C5E0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x1b557256f50) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b551265fd0 is calling an instrumented method <function Refine.get_response at 0x000001B537B57E20>. The path of this call may be incorrect.
Guessing path of new object is ap

✅ What are the limitations or gaps identified in the document?


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1b551265a90 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x000001B5390C5620>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1b557257050) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b551265fd0 is calling an instrumented method <function CompactAndRefine.get_response at 0x000001B53725C5E0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x1b557256f50) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1b551265fd0 is calling an instrumented method <function Refine.get_response at 0x000001B537B57E20>. The path of this call may be incorrect.
Guessing path of new object is ap

✅ What recommendations or next steps does the document propose?


### 5 - Sentence Window s=10

In [15]:
# from llama_index.llms import OpenAI

sentence_index_10 = build_sentence_window_index(
    pdf,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0),
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=10,
    save_dir="sentence_index_10",
)

sentence_window_engine_10 = get_sentence_window_query_engine(
    sentence_index_10
)

tru_recorder_10 = get_prebuilt_trulens_recorder(
    sentence_window_engine_10,
    app_id='Sentence Window s=10'
)

run_evals(eval_questions, tru_recorder_10, sentence_window_engine_10)

✅ What is the main theme or topic of this document?
✅ Can you summarize the key points made in this document?
✅ What are the primary arguments or claims presented?
✅ Are there any notable facts or figures mentioned? If so, what are they?
✅ Can you provide an example or case study mentioned in the document?
✅ What conclusions are drawn in this document?
✅ Are there any contradictions or points of debate within the document?
✅ What background or contextual information is provided?
✅ What are the limitations or gaps identified in the document?
✅ What recommendations or next steps does the document propose?


### 6 - Sentence Window s=20

In [16]:
# from llama_index.llms import OpenAI

sentence_index_20 = build_sentence_window_index(
    pdf,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0),
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=20,
    save_dir="sentence_index_20",
)

sentence_window_engine_20 = get_sentence_window_query_engine(
    sentence_index_20
)

tru_recorder_20 = get_prebuilt_trulens_recorder(
    sentence_window_engine_20,
    app_id='Sentence Window s=20'
)

run_evals(eval_questions, tru_recorder_20, sentence_window_engine_20)

✅ What is the main theme or topic of this document?
✅ Can you summarize the key points made in this document?
✅ What are the primary arguments or claims presented?
✅ Are there any notable facts or figures mentioned? If so, what are they?
✅ Can you provide an example or case study mentioned in the document?
✅ What conclusions are drawn in this document?
✅ Are there any contradictions or points of debate within the document?
✅ What background or contextual information is provided?
✅ What are the limitations or gaps identified in the document?
✅ What recommendations or next steps does the document propose?


### 7 - Sentence Window s=30

In [ ]:
# from llama_index.llms import OpenAI

# sentence_index_30 = build_sentence_window_index(
#     pdf,
#     llm=OpenAI(model="gpt-3.5-turbo", temperature=0),
#     embed_model="local:BAAI/bge-small-en-v1.5",
#     sentence_window_size=30,
#     save_dir="sentence_index_30",
# )

# sentence_window_engine_30 = get_sentence_window_query_engine(
#     sentence_index_30
# )

# tru_recorder_30 = get_prebuilt_trulens_recorder(
#     sentence_window_engine_30,
#     app_id='Sentence Window s=30'
# )

# run_evals(eval_questions, tru_recorder_30, sentence_window_engine_30)

### 8 - Sentence Window s=40

In [ ]:
# # from llama_index.llms import OpenAI

# sentence_index_40 = build_sentence_window_index(
#     pdf,
#     llm=OpenAI(model="gpt-3.5-turbo", temperature=0),
#     embed_model="local:BAAI/bge-small-en-v1.5",
#     sentence_window_size=40,
#     save_dir="sentence_index_40",
# )

# sentence_window_engine_40 = get_sentence_window_query_engine(
#     sentence_index_40
# )

# tru_recorder_40 = get_prebuilt_trulens_recorder(
#     sentence_window_engine_40,
#     app_id='Sentence Window s=40'
# )

# run_evals(eval_questions, tru_recorder_40, sentence_window_engine_40)

✅ What is the main theme or topic of this document?
✅ Can you summarize the key points made in this document?
✅ What are the primary arguments or claims presented?
✅ Are there any notable facts or figures mentioned? If so, what are they?
✅ Can you provide an example or case study mentioned in the document?
✅ What conclusions are drawn in this document?
✅ Are there any contradictions or points of debate within the document?
✅ What background or contextual information is provided?
✅ What are the limitations or gaps identified in the document?
✅ What recommendations or next steps does the document propose?


### 9 - Sentence Window s=50

In [17]:
from llama_index.llms import OpenAI

sentence_index_50 = build_sentence_window_index(
    pdf,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0),
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=50,
    save_dir="sentence_index_50",
)

sentence_window_engine_50 = get_sentence_window_query_engine(
    sentence_index_50
)

tru_recorder_50 = get_prebuilt_trulens_recorder(
    sentence_window_engine_50,
    app_id='Sentence Window s=50'
)

run_evals(eval_questions, tru_recorder_50, sentence_window_engine_50)

✅ What is the main theme or topic of this document?
✅ Can you summarize the key points made in this document?
✅ What are the primary arguments or claims presented?
✅ Are there any notable facts or figures mentioned? If so, what are they?
✅ Can you provide an example or case study mentioned in the document?
✅ What conclusions are drawn in this document?
✅ Are there any contradictions or points of debate within the document?
✅ What background or contextual information is provided?
✅ What are the limitations or gaps identified in the document?
✅ What recommendations or next steps does the document propose?


### 10 - Sentence Window s=70

In [ ]:
# from llama_index.llms import OpenAI

# sentence_index_70 = build_sentence_window_index(
#     pdf,
#     llm=OpenAI(model="gpt-3.5-turbo", temperature=0),
#     embed_model="local:BAAI/bge-small-en-v1.5",
#     sentence_window_size=70,
#     save_dir="sentence_index_70",
# )

# sentence_window_engine_70 = get_sentence_window_query_engine(
#     sentence_index_70
# )

# tru_recorder_70 = get_prebuilt_trulens_recorder(
#     sentence_window_engine_70,
#     app_id='Sentence Window s=70'
# )

# run_evals(eval_questions, tru_recorder_70, sentence_window_engine_70)

### 11 - Sentence Window s=100

In [ ]:
# from llama_index.llms import OpenAI

sentence_index_100 = build_sentence_window_index(
    pdf,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0),
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=100,
    save_dir="sentence_index_100",
)

sentence_window_engine_100 = get_sentence_window_query_engine(
    sentence_index_100
)

tru_recorder_100 = get_prebuilt_trulens_recorder(
    sentence_window_engine_100,
    app_id='Sentence Window s=100'
)

run_evals(eval_questions, tru_recorder_100, sentence_window_engine_100)

In [ ]:
# Vizualize evaluation results on each concept
tru.run_dashboard()

In [ ]:
tru.stop_dashboard()